## This notebook processes the training images to augment the dataset
#### (The processing includes resizing, and augmentation of the images (by applying rotations, flipping etc.))

## 1) Setting up access to watson data assets

#### Insert project token

In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='fbb84b2e-f008-4583-851a-fb0a45856e38', project_access_token='p-651e508bfb830754df7129d064d40f7a5f292653')
pc = project.project_context


#### Get Project Data Assets

In [8]:
#CHECK THIS CELL BEFORE RUNNING

data_assets = project.get_assets(asset_type='data_asset')
# print(data_assets)

# Enter the letters to help further filtering of files: Eg. "BA", or "_go_", or "Rebar"
searchfilter = "Rebar"

filenames = [file['name'] for file in data_assets if searchfilter in file['name']]
print(filenames)

['Rebar Go Cropped.zip', 'Rebar No Go Cropped.zip']


#### Extract Zip Files

In [10]:
import zipfile
import os

cwd = "/home/dsxuser/work"

def get_zip(file_name):
    '''
    file_name = Name of zip file you want to download from object storage
    '''
    os.chdir(cwd)
    folder = file_name.split(".")[0]
    os.mkdir(folder)
    os.chdir(folder)

    try:
        fobj = open(file_name, "wb")
        fobj.write(project.get_file(file_name).read()) 
        fobj.close()
        z = zipfile.ZipFile(file_name)
        z.extractall()
    except Exception as e:
        print(Exception,e)
    else:
        print('Files downloaded successfully') 

## EXTRACT ALL ZIP FOLDERS
for filename in filenames:
    get_zip(filename)

## 2) Process the Images

#### Processing steps: Resize, Horizontal flip, Vertical Flip, Rotation by 15,30,45,60,74,90 degrees

In [41]:
import numpy as np
import shutil
import math
import cv2
import os


##ROTATE IMAGES
def rotateImages(imgs): #NOTE: imgs needs to be a list of images
    angles = [15,30,45,60,75,90] #Angles of rotation
    rotated_images = []
    
    for img in imgs:   
        height, width = img.shape[:2]
        image_center = (width / 2, height / 2)
        scale = 1
        
        for angle in angles:
            
            rotation_mat = cv2.getRotationMatrix2D(image_center, angle, scale)

            radians = math.radians(angle)
            sin = math.sin(radians)
            cos = math.cos(radians)
            bound_w = int((height * abs(sin)) + (width * abs(cos)))
            bound_h = int((height * abs(cos)) + (width * abs(sin)))

            rotation_mat[0, 2] += ((bound_w / 2) - image_center[0])
            rotation_mat[1, 2] += ((bound_h / 2) - image_center[1])

            rotated_images.append(cv2.warpAffine(img, rotation_mat, (bound_w, bound_h),
                                                borderMode=cv2.BORDER_CONSTANT,
                                                borderValue=(255,255,255)))
    return rotated_images


## SAVE IMAGES TO ANOTHER FOLDER

## imgs = images to be saved, newdirname = new directory name to save the files, filepath is the image path

def saveImages(imgs,newdirname,filepath):
    cwd = "/home/dsxuser/work" #Current Working Directory
    for i,img in enumerate(imgs):
        filename    = os.path.basename(filepath).split(".")[0]
        filename    = "T_"+filename+"_"+str(i)+".png" #File name logic
        
        
        newfilepath  =  cwd+"/"+newdirname+"/"+filename
        cv2.imwrite(newfilepath, img)
        
## UPLOAD DATA TO CLOUD OBJECT STORAGE (COS)
def uploadData(file):
    os.chdir("/home/dsxuser/work")
    dirpath = cwd+"/"+file
    print(dirpath)
    shutil.make_archive(dirpath,'zip',dirpath) #ZIP FILES
    
    #Save data to COS
    name = file+".zip"
    file_data=open(name, 'rb')
    project.save_data(data=file_data.read(),file_name=name,overwrite=True)
    file_data.close()
    
       
def processImages(dirpath): #dirpath = directory path of images 
    
    newdirname = "BA_"+"T_"+os.path.basename(dirpath) #Setup folder naming logic as needed
    newdirpath = cwd+"/"+newdirname
    os.chdir(cwd)
    os.mkdir(newdirpath)
    
    files = [dirpath+"/"+item for item in os.listdir(dirpath) if ".png" in item]
    files = np.array(files)
    
    for filepath in files:
        img = cv2.imread(filepath)

        # 1) Flip Image -Vertical
        flip_v = np.flip(img,0)

        # 3) Flip Image -Horizontal
        flip_h = np.flip(img,0)

#         # 4) Rotate Image (by affine transform)
        imgs = [img,flip_v,flip_h]
        rotated_images = rotateImages(imgs)

        transformed_images = [*imgs,*rotated_images]
        
        #Save Images to Local Storage
        saveImages(transformed_images,newdirname,filepath)
    
    #Upload data to Cloud Object Storage
    uploadData(newdirname)

    print("SUCCESS")
        
        

In [1]:
#list of folders to process

alldirs = [file for file in os.listdir(cwd) if ".zip" not in file] 

for dirname in alldirs:
    dirpath = cwd+"/"+dirname
    processImages(dirpath)